In [112]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 

In [218]:
#read in df (filtered) and drop unncessary columns

filtered_df = pd.read_csv("data_original_full_filteredasignees.csv")
filtered_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace=True)

In [189]:
#drop ground truth columns

filtered_df = filtered_df[filtered_df['assignee_masked']!=1000]

In [190]:
#creating unique_task column and dropping seg_id and ID

filtered_df['unique_task']=filtered_df['seg_id'].astype(str)+'_'+filtered_df['ID'].astype(str)
filtered_df.drop(['seg_id', 'ID'], axis = 1, inplace=True)

In [192]:
#groupby unique task and get mean and std for each task, then threshold by 3std rule

unique_duration_df = filtered_df.groupby(['unique_task']).agg(['mean','std'])['duration']
unique_duration_df['threshold']=unique_duration_df['mean']+3*unique_duration_df['std']
unique_duration_df

/var/folders/nq/wg8jmjws6rzcz7mzf1f4z0880000gn/T/ipykernel_38243/860348785.py:3: FutureWarning: ['closed', 'opened', 'status', 'created', 'decision'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  unique_duration_df = filtered_df.groupby(['unique_task']).agg(['mean','std'])['duration']
/var/folders/nq/wg8jmjws6rzcz7mzf1f4z0880000gn/T/ipykernel_38243/860348785.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_duration_df['threshold']=unique_duration_df['mean']+3*unique_duration_df['std']


,mean,std,threshold
unique_task,,,
864691134964463647_96,75.615385,60.315886,256.563044
864691135081758967_69,120.461538,62.384761,307.615823
864691135081758967_70,141.076923,172.192665,657.654918
864691135081873399_123,160.038462,127.054470,541.201873
864691135291777718_169,160.384615,118.259402,515.162821
...,...,...,...
864691136808588398_55,306.576923,292.631669,1184.471931
864691137196956225_151,134.923077,102.615368,442.769182
864691137196956225_152,126.961538,141.367742,551.064764


In [194]:
#create a new threshold column in filtered_df to match up all thresholds to unique tasks
filtered_df['threshold']=filtered_df['unique_task'].apply(unique_duration_df['threshold'].get)

In [197]:
#replace any values over the threshold for the unique task with the threshold
filtered_df.loc[filtered_df['duration'] > filtered_df['threshold'], 'duration'] = filtered_df['threshold']

In [200]:
filtered_df

,closed,opened,duration,status,created,decision,score,assignee_masked,unique_task,threshold
0,2022-01-03 15:25:29.708999936,2022-01-03 15:24:20.075000064,68.0,closed,2021-12-30 13:29:06.153000000,no,1.0,1,864691135725642047_0,2188.906494
1,2022-01-03 15:38:13.128000000,2022-01-03 15:30:20.956000000,470.0,closed,2021-12-30 13:29:06.153000000,yes,0.0,2,864691135725642047_0,2188.906494
2,2022-01-03 15:22:43.824999936,2022-01-03 14:30:59.647000064,58.0,closed,2021-12-30 13:29:06.153000000,no,1.0,3,864691135725642047_0,2188.906494
3,2022-01-03 15:21:44.276000000,2022-01-03 15:17:47.611000064,235.0,closed,2021-12-30 13:29:06.153000000,no,1.0,4,864691135725642047_0,2188.906494
4,2022-01-03 15:23:48.920999936,2022-01-03 15:18:52.494000128,216.0,closed,2021-12-30 13:29:06.153000000,no,1.0,5,864691135725642047_0,2188.906494
...,...,...,...,...,...,...,...,...,...,...
2384,2022-01-04 18:51:31.833999872,2022-01-04 18:50:14.358000128,75.0,closed,2022-01-04 13:00:39.713000000,no,1.0,26,864691135366968569_171,308.122163
2385,2022-01-04 21:03:13.839000064,2022-01-04 21:01:45.948000000,86.0,closed,2022-01-04 13:00:39.713000000,no,1.0,21,864691135366968569_171,308.122163
2386,2022-01-04 20:08:50.419000064,2022-01-04 20:08:12.532000000,37.0,closed,2022-01-04 13:00:39.713000000,no,1.0,22,864691135366968569_171,308.122163
2387,2022-01-04 19:21:20.531000064,2022-01-04 19:20:25.012999936,54.0,closed,2022-01-04 13:00:39.713000000,no,1.0,23,864691135366968569_171,308.122163


In [201]:
#Grouped filtered_df by assignees --> summed columns to get total duration and score

assignee_df=filtered_df.groupby(['assignee_masked']).sum()

#Determined number of rows belonging to each assignee and appended it to assignee_df

assignee_df['task_num']=filtered_df['assignee_masked'].value_counts().sort_index()

#Found avg accuracy_score by dividing cumulative score by task_num
assignee_df['average_accuracy']=assignee_df['score']/assignee_df['task_num']

#Found avg duration by dividing cumulative time by task_num
assignee_df['average_duration']=assignee_df['duration']/assignee_df['task_num']

assignee_df

/var/folders/nq/wg8jmjws6rzcz7mzf1f4z0880000gn/T/ipykernel_38243/3506429052.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  assignee_df=filtered_df.groupby(['assignee_masked']).sum()


,duration,score,threshold,task_num,average_accuracy,average_duration
assignee_masked,,,,,,
1,15422.810141,67.5,55543.551555,90,0.750000,171.364557
2,14452.000000,54.5,55543.551555,90,0.605556,160.577778
3,9562.107915,77.0,55543.551555,90,0.855556,106.245643
4,7496.000000,77.5,55543.551555,90,0.861111,83.288889
5,13117.000000,81.5,55543.551555,90,0.905556,145.744444
6,12562.000000,68.0,55543.551555,90,0.755556,139.577778
7,11883.868425,74.0,55543.551555,90,0.822222,132.042983
8,10153.124185,72.5,55543.551555,90,0.805556,112.812491
9,8855.000000,77.5,55543.551555,90,0.861111,98.388889


In [202]:
#Dropped unnecessary columns --> converted df to dictionary

metric_dict=assignee_df.drop(['duration','score','threshold', 'task_num'], axis=1).to_dict(orient='index')

In [203]:
metric_dict.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26])

In [204]:
metric_dict.values()

dict_values([{'average_accuracy': 0.75, 'average_duration': 171.36455711725694}, {'average_accuracy': 0.6055555555555555, 'average_duration': 160.57777777777778}, {'average_accuracy': 0.8555555555555555, 'average_duration': 106.24564349474925}, {'average_accuracy': 0.8611111111111112, 'average_duration': 83.28888888888889}, {'average_accuracy': 0.9055555555555556, 'average_duration': 145.74444444444444}, {'average_accuracy': 0.7555555555555555, 'average_duration': 139.57777777777778}, {'average_accuracy': 0.8222222222222222, 'average_duration': 132.04298250036143}, {'average_accuracy': 0.8055555555555556, 'average_duration': 112.81249094425428}, {'average_accuracy': 0.8611111111111112, 'average_duration': 98.38888888888889}, {'average_accuracy': 0.8944444444444445, 'average_duration': 223.61452652080192}, {'average_accuracy': 0.7388888888888889, 'average_duration': 65.72222222222223}, {'average_accuracy': 0.7, 'average_duration': 207.57096886268116}, {'average_accuracy': 0.716666666666

In [217]:
#Compiling into a function

def extract_metrics(filtered_df):
    '''
        filtered_df: Contains columns: duration, score, seg_id, ID
        Return: nested dictionary as Subject:Metrics format
    '''
    #drop ground truth columns

    filtered_df = filtered_df[filtered_df['assignee_masked']!=1000]

    #create unique_task column and drop seg_id and ID

    filtered_df['unique_task']=filtered_df['seg_id'].astype(str)+'_'+filtered_df['ID'].astype(str)

    filtered_df.drop(['seg_id', 'ID'], axis = 1, inplace=True)

    #groupby unique task and get mean and std for each task, then threshold by 3std rule

    unique_duration_df = filtered_df.groupby(['unique_task']).agg(['mean','std'])['duration']
    unique_duration_df['threshold']=unique_duration_df['mean']+3*unique_duration_df['std']

    #create a new threshold column in filtered_df to match up all thresholds to unique tasks

    filtered_df['threshold']=filtered_df['unique_task'].apply(unique_duration_df['threshold'].get)

    #replace any values over the threshold for the unique task with the threshold

    filtered_df.loc[filtered_df['duration'] > filtered_df['threshold'], 'duration'] = filtered_df['threshold']

    #Group filtered_df by assignees --> sum columns to get total duration and score

    assignee_df=filtered_df.groupby(['assignee_masked']).sum()

    #Determine number of rows belonging to each assignee and append it to assignee_df

    assignee_df['task_num']=filtered_df['assignee_masked'].value_counts().sort_index()

    #Find average accuracy_score by dividing cumulative score by task_num

    assignee_df['average_accuracy']=assignee_df['score']/assignee_df['task_num']

    #Find avg duration by dividing cumulative time by task_num

    assignee_df['average_duration']=assignee_df['duration']/assignee_df['task_num']

    #convert df to dictionary

    metric_dict=assignee_df.drop(['duration','score','threshold', 'task_num'], axis=1).to_dict(orient='index')
    
    return metric_dict;


In [219]:
extract_metrics(filtered_df)

/var/folders/nq/wg8jmjws6rzcz7mzf1f4z0880000gn/T/ipykernel_38243/1924552765.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['unique_task']=filtered_df['seg_id'].astype(str)+'_'+filtered_df['ID'].astype(str)
/var/folders/nq/wg8jmjws6rzcz7mzf1f4z0880000gn/T/ipykernel_38243/1924552765.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['seg_id', 'ID'], axis = 1, inplace=True)
/var/folders/nq/wg8jmjws6rzcz7mzf1f4z0880000gn/T/ipykernel_38243/1924552765.py:20: FutureWarning: ['closed', 'opened', 'status', 'created', 'decision'] did n

{1: {'average_accuracy': 0.75, 'average_duration': 171.36455711725694},
 2: {'average_accuracy': 0.6055555555555555,
  'average_duration': 160.57777777777778},
 3: {'average_accuracy': 0.8555555555555555,
  'average_duration': 106.24564349474925},
 4: {'average_accuracy': 0.8611111111111112,
  'average_duration': 83.28888888888889},
 5: {'average_accuracy': 0.9055555555555556,
  'average_duration': 145.74444444444444},
 6: {'average_accuracy': 0.7555555555555555,
  'average_duration': 139.57777777777778},
 7: {'average_accuracy': 0.8222222222222222,
  'average_duration': 132.04298250036143},
 8: {'average_accuracy': 0.8055555555555556,
  'average_duration': 112.81249094425428},
 9: {'average_accuracy': 0.8611111111111112,
  'average_duration': 98.38888888888889},
 10: {'average_accuracy': 0.8944444444444445,
  'average_duration': 223.61452652080192},
 11: {'average_accuracy': 0.7388888888888889,
  'average_duration': 65.72222222222223},
 12: {'average_accuracy': 0.7, 'average_duration'